<h1>Merge RDD rows by Start/End Pattern</h1>

<p>Concatenate block of multiple lines based on Start/End patterns</p>
<p>Example: https://stackoverflow.com/questions/50865236/convert-multiple-rdd-rows-into-one-row-in-pyspark</p>

<p>In this case merging lines between <b>STARTING</b> and <b>STOP</b> marks. Since the same block (from STARTING to STOP) can be processed in multiple partitions, it's not easy to handle the Start and End marks at the same time. </p>

<p>We can handle this task in two steps:</p>

<ol>
 <li>Split the block with the '<b>STOP</b>' block by functions like <b>fold()</b>, <b>aggregate()</b>, <b>reduce()</b>
    Mark STARTING with a preceeding NUL character: '<b>\0</b>' </li>
 <li>Reload data into RDD and clean the unrelated text before the <b>NUL</b> + '<b>STARTING</b>' using <b>map()</b> function </li>
</ol>

<p>Below code was tested under ipython:</p>

<p>Sample text:</p>
<pre>
skip0 STARTING
skip0
STARTING |1|TH|TGG|132|8|T|Fall|
EVENT 1|56|HT|JUP||||||||
EVENT 2|BHT|987|231|||||||||||||||||
STOP|HFR|0.5|90|
skip1
skip1
skip1
STARTING |8|TH|TGG|12|8|T|Fall|
EVENT 1|6|HT|UP||||||||
EVENT 2|BT|987|31|||||||||||||||||
STOP|FR|0.5|90|
skip2
skip2
</pre>


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder                   \
                    .master("local[2]")        \
                    .appName("pyspark-test1")  \
                    .getOrCreate()
sc = spark.sparkContext

In [ ]:
# read the text file and check the partitions
rdd = sc.textFile("/test/pyspark/merge-1.txt", minPartitions=20)
rdd.glom().collect()

In [ ]:
# fold function to merge data in and between partitions
# Note: we preceeded a NUL char before the 'STARTING' to split the unwanted text later
def merge(x, y):
    if type(y) is list:
        x[-1] += y[0]
        x = x + y[1:]
    else:
        if y.startswith('STARTING'):
            x[-1] += '\0' + y
        else:
            x[-1] += y
        if y.startswith('STOP'):
            x.append('')
    return x

In [ ]:
# the result from the first step Terminate the block at 'STOP' line while keeping messy before the 'Start' line
rdd.fold([''], merge)

In [ ]:
# read the data into RDD again and remove the precedding text with map() function
rdd1 = sc.parallelize(rdd.fold([''], merge)[:-1])
rdd1.map(lambda x: x[x.find('\0STARTING')+1:]).collect()